# OpenDSPL Interpreter Architecture
## Overview

**The code here generate illustrations, it is not part of the specification.**

In [44]:
from graphviz import Digraph
dot = Digraph(comment="OpenDSPL Interpreter Architecture")
dot.node('S', 'source', shape='note')
dot.node('L', 'libs', shape='box3d')
dot.node('P', 'PRE-PROCESSOR')
dot.edges(['SP', 'LP'])
dot.node('par', 'PARSER')
dot.edge('P', 'par', label='code')
dot.node('I', 'INTERPRETER')
dot.edge('par', 'I', 'AST')
dot.node('C', 'compressed identifier\nhash table', shape='box3d')
dot.node('G', 'signal graph', shape='box3d')
dot.node('N', 'input', shape='note')
dot.node('M', 'VIRTUAL MACHINE')
dot.edges(['IC', 'IG', 'CM', 'GM', 'NM'])
dot.node('out', 'output', shape='note')
dot.edge('M', 'out')
dot

## Interpreter Design
The interpreter has the task of traversing the validated AST produced by the parser and
converting it into useful datastructures for the virtual machine. The datastructures it
produces are a hash-table storing identifiers and their associated expressions, and a
graph that describes how the output of each identifier being evaluated relates to every
other identifier.

The way this is performed is by first creating a hash-table for the identifiers, then
reducing it in size as much as possible, by substituting identifiers with their
associated expressions, until the last remaining identifiers are all un-reduceable.
This is called identifier compression.

After this step, a graph is generated, with a node for each surviving identifier, with
an in-going connection from each undefined identifier it depends from and an out-going
connection for each identifier that depends on it.

Finally, every graph that doesn't end in a sink should raise a warning, every graph
that has no source should raise an error.

## Virtual Machine Design

The first step in the execution of the code is building a source table. For each graph
all the sources are identified and the files they refer to are opened. Then the sink
table is built. For each sink in the graph, the related files are opened.

Each source in the source table has a specified sample rate. The main clock rate that
will drive all sources is derived from the individual sample rates by taking their
least common multiple (LCM). Then the "clock period" for each source's clock (the
number of main samples from one source sample to the next. This step is necessary for
clock synchronization. Each source sends an event out every time the clock period has
elapsed. Each source has thus a counter for counting main clock events.